In [1]:
import pandas as pd
import requests
import pymongo

In [14]:
df = pd.DataFrame(
    data={
        'name': ["black", "blue", "brown", "gray", "green", "pink", "purple", "red", "white", "yellow"],
        'number': [40, 145, 123, 77, 87, 47, 75, 82, 63, 68]
    }
)
l = []
d = df.T.to_dict()
for item in d:
    l.append(d[item])
    
print(l)

[{'name': 'black', 'number': 40}, {'name': 'blue', 'number': 145}, {'name': 'brown', 'number': 123}, {'name': 'gray', 'number': 77}, {'name': 'green', 'number': 87}, {'name': 'pink', 'number': 47}, {'name': 'purple', 'number': 75}, {'name': 'red', 'number': 82}, {'name': 'white', 'number': 63}, {'name': 'yellow', 'number': 68}]


In [4]:
base_url = 'https://pokeapi.co/api/v2/'
color_url = base_url + 'pokemon-color/'
evolution_url = base_url + 'evolution-chain/?offset=0&limit=419'

#get porkemon color

In [3]:
pokemon_color = requests.get(color_url).json()['results']
pokemon_color

[{'name': 'black', 'url': 'https://pokeapi.co/api/v2/pokemon-color/1/'},
 {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/pokemon-color/2/'},
 {'name': 'brown', 'url': 'https://pokeapi.co/api/v2/pokemon-color/3/'},
 {'name': 'gray', 'url': 'https://pokeapi.co/api/v2/pokemon-color/4/'},
 {'name': 'green', 'url': 'https://pokeapi.co/api/v2/pokemon-color/5/'},
 {'name': 'pink', 'url': 'https://pokeapi.co/api/v2/pokemon-color/6/'},
 {'name': 'purple', 'url': 'https://pokeapi.co/api/v2/pokemon-color/7/'},
 {'name': 'red', 'url': 'https://pokeapi.co/api/v2/pokemon-color/8/'},
 {'name': 'white', 'url': 'https://pokeapi.co/api/v2/pokemon-color/9/'},
 {'name': 'yellow', 'url': 'https://pokeapi.co/api/v2/pokemon-color/10/'}]

In [11]:
pokemon_color_data = []
for item in pokemon_color:
    url = item['url']
    name = item['name']
    r = requests.get(url).json()['pokemon_species']
    pokemon_color_data.append({'name': name, 'number': len(r)})

pokemon_color_df = pd.DataFrame(data=pokemon_color_data)

In [12]:
pokemon_color_df

,name,number
0,black,40
1,blue,145
2,brown,123
3,gray,77
4,green,87
5,pink,47
6,purple,75
7,red,82
8,white,63
9,yellow,68


# evolution-chain

In [29]:
evolution_chain = requests.get(evolution_url).json()['results']

In [35]:
def evolve_pokemon(url):
    url = url
    d = requests.get(url).json()['chain']
    chain = []
    chain.append(d['species']['name'])
    evolve = d['evolves_to']
    while(evolve != []):
        chain.append(evolve[0]['species']['name'])
        evolve = evolve[0]['evolves_to']
    return chain

In [48]:
evolution_data = []
for index, item in enumerate(evolution_chain):
    url = item['url']
    pokemon = evolve_pokemon(url)
    data = {'evl_id': index, 'chain': pokemon, 'stage': len(pokemon)}
    evolution_data.append(data)

In [49]:
# save raw data to my mongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.pokemon_db
# create colletions
col = db['pokemon_evolve']
col.insert_many(evolution_data)

In [53]:
evolution_df = pd.DataFrame(list(col.find()))

In [57]:
two_stage = evolution_df.loc[evolution_df['stage'] == 2][['evl_id', 'chain']]

In [58]:
two_stage

,evl_id,chain
6,6,"[rattata, raticate]"
7,7,"[spearow, fearow]"
8,8,"[ekans, arbok]"
10,10,"[sandshrew, sandslash]"
14,14,"[vulpix, ninetales]"
18,18,"[paras, parasect]"
19,19,"[venonat, venomoth]"
20,20,"[diglett, dugtrio]"
21,21,"[meowth, persian]"
22,22,"[psyduck, golduck]"
